
# 1D Poisson Problem with Finite Differences (TP)

We solve the boundary value problem
\[-u''(x) = f(x), \quad x\in(0,1), \qquad u(0)=u(1)=0.\]

Goals:
- Implement the **second-order centered** finite-difference scheme.
- Assemble the **tridiagonal** matrix and solve the linear system.
- Test several right-hand sides \(f\), compare to **exact** solutions when known (cases 1 & 2), otherwise to a **fine-grid reference** (cases 3 & 4).
- Study **convergence**: plot \(\|u-u_\star\|_\infty\) vs \( \Delta x \) on log–log (order ≈ slope).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

%matplotlib inline



## Right-hand sides and exact solutions


In [ ]:
def f_rhs(x, ind):
    x = np.asarray(x)
    if ind == 1:
        return np.ones_like(x)
    elif ind == 2:
        return np.sin(np.pi*x)
    elif ind == 3:
        return ((1/4 <= x) & (x < 3/4)).astype(float)
    elif ind == 4:
        return np.exp(-(x-0.5)**2)
    else:
        raise ValueError("ind must be 1..4")

def exact_u(x, ind):
    x = np.asarray(x)
    if ind == 1:
        return x*(1-x)/2.0
    elif ind == 2:
        return np.sin(np.pi*x)/(np.pi**2)
    else:
        return np.zeros_like(x)  # not used



## Finite-difference solver (second-order centered)


In [ ]:
def solve_poisson_1d(J, f_handle, bc0=0.0, bc1=0.0, sparse=True):
    """Solve -u'' = f on (0,1) with u(0)=bc0, u(1)=bc1 using J interior points.
    Returns (x_full, u_full) including endpoints.
    """
    x = np.linspace(0.0, 1.0, J+2)     # includes endpoints
    dx = x[1]-x[0]
    xi = x[1:-1]                       # interior points

    main = 2.0*np.ones(J)
    off  = -1.0*np.ones(J-1)
    if sparse:
        A = diags([off, main, off], [-1,0,1], shape=(J,J), dtype=float) / (dx*dx)
        b = f_handle(xi).astype(float)
        b[0]  += bc0 / (dx*dx)
        b[-1] += bc1 / (dx*dx)
        u_int = spsolve(A, b)
    else:
        A = np.diag(main)
        A += np.diag(off, -1) + np.diag(off, 1)
        A /= (dx*dx)
        b = f_handle(xi).astype(float)
        b[0]  += bc0 / (dx*dx)
        b[-1] += bc1 / (dx*dx)
        u_int = np.linalg.solve(A, b)

    u = np.empty(J+2)
    u[0], u[-1] = bc0, bc1
    u[1:-1] = u_int
    return x, u



## Visual check (all 4 cases)


In [ ]:
def reference_solution(ind, J_ref=8192):
    xr, ur = solve_poisson_1d(J_ref, lambda x: f_rhs(x, ind))
    return xr, ur

J = 100
for ind in [1,2,3,4]:
    x, u = solve_poisson_1d(J, lambda x: f_rhs(x, ind))
    plt.figure()
    plt.plot(x, u, '+', label="numerical (FD)")
    if ind in [1,2]:
        xf = np.linspace(0,1,400)
        plt.plot(xf, exact_u(xf, ind), label="exact")
    else:
        xr, ur = reference_solution(ind)
        plt.plot(xr, ur, label="reference (fine FD)")
    plt.xlabel("x"); plt.ylabel("u(x)")
    plt.title(f"Poisson 1D FD — case ind={ind}, J={J}")
    plt.legend(); plt.show()



## Convergence study
We compute \(\|u-u_*\|_\infty\) versus \(\Delta x\). Order-2 should be observed.


In [ ]:
import pandas as pd

def max_error(ind, Js):
    rows = []
    for J in Js:
        x, u = solve_poisson_1d(J, lambda x: f_rhs(x, ind))
        dx = x[1]-x[0]
        if ind in [1,2]:
            uex = exact_u(x, ind)
        else:
            xr, ur = reference_solution(ind, J_ref=16384)
            uex = np.interp(x, xr, ur)
        err = np.max(np.abs(u-uex))
        rows.append({"J": J, "dx": dx, "err_inf": err})
    return pd.DataFrame(rows)

def loglog_plot(df, title):
    plt.figure()
    plt.loglog(df["dx"], df["err_inf"], marker="o", label="FD error")
    dx = df["dx"].values
    e2 = (df["err_inf"].values[-1]/(dx[-1]**2))*(dx**2)
    e1 = (df["err_inf"].values[-1]/(dx[-1]))*(dx)
    plt.loglog(dx, e1, "--", label="order 1")
    plt.loglog(dx, e2, "--", label="order 2")
    plt.gca().invert_xaxis()
    plt.xlabel("Δx"); plt.ylabel("||u - u*||_∞")
    plt.title(title); plt.legend(); plt.show()

Js = [10, 20, 40, 80, 160, 320, 640]
for ind in [1,2,3,4]:
    dfc = max_error(ind, Js)
    display(dfc.head(3))
    loglog_plot(dfc, f"Convergence (ind={ind})")
